In [ ]:
import json
import cv2
import numpy as np

# Patch metadata
row, col = 39, 18
patch_size = 1024
overlap = 0.0625  # 6.25%
step_size = int(patch_size * (1 - overlap))  # 960 pixels
# step_size = 900

# Calculate patch offset
x_offset = col * step_size  # 9600
y_offset = row * step_size  # 0
patch_bounds = (x_offset, y_offset, x_offset + patch_size, y_offset + patch_size)  # (9600, 0, 10624, 1024)

# Load GeoJSON from full WSI inference
geojson_path = "/media/network/hdd/santosh/cellvit_preprocessing_gall-bladder/patches/S2503154_A8_tiff_pyramid/cell_detection/cells.geojson"
with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)

# Function to calculate centroid of a polygon (first ring only for simplicity)
def get_centroid(coords):
    coords = np.array(coords)
    x, y = coords[:, 0], coords[:, 1]
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    return centroid_x, centroid_y

# Filter and transform coordinates
patch_features = []
for feature in geojson_data:  # Directly iterate over the list
    if feature["geometry"]["type"] == "MultiPolygon":
        # Use the first polygon ring for centroid calculation
        coords = feature["geometry"]["coordinates"][0][0]  # First MultiPolygon, first ring
        centroid_x, centroid_y = get_centroid(coords)
        
        # Check if centroid is within patch bounds
        if (patch_bounds[0] <= centroid_x < patch_bounds[2] and 
            patch_bounds[1] <= centroid_y < patch_bounds[3]):
            # Transform all rings to patch-local coordinates
            local_coords = []
            for polygon in feature["geometry"]["coordinates"]:
                transformed_rings = []
                for ring in polygon:
                    transformed_ring = [[x - x_offset, y - y_offset] for x, y in ring]
                    transformed_rings.append(transformed_ring)
                local_coords.append(transformed_rings)
            feature["geometry"]["coordinates"] = local_coords  # Update coordinates
            patch_features.append(feature)

# # Save filtered GeoJSON (optional)
# patch_geojson = patch_features  # Already a list, no need for FeatureCollection wrapper
# with open("patch_segmentation.geojson", "w") as f:
#     json.dump(patch_geojson, f)

# Load patch image
patch_image_path = "/media/network/hdd/santosh/cellvit_preprocessing_gall-bladder/patches/S2503154_A8_tiff_pyramid/patches/S2503154_A8_tiff_pyramid_39_18.png"
patch = cv2.imread(patch_image_path)
print(f"Patch image shape: {patch.shape}")

# Overlay nuclei with cell type colors
for feature in patch_features:
    # Extract cell type and color from properties
    try:
        cell_type = feature["properties"]["classification"]["name"]
        rgb_color = feature["properties"]["classification"]["color"]  # RGB format: [R, G, B]
        # Convert RGB to BGR for OpenCV
        bgr_color = (rgb_color[2], rgb_color[1], rgb_color[0])
    except (KeyError, IndexError):
        # Default color (red) if classification or color is missing
        cell_type = "Unknown"
        bgr_color = (0, 0, 255)  # Red in BGR

    # Print cell type for debugging
    print(f"Cell type: {cell_type}, Color (BGR): {bgr_color}")

    for polygon in feature["geometry"]["coordinates"]:  # Iterate over MultiPolygon
        coords = np.array(polygon[0], dtype=np.int32)  # First ring of each polygon
        # Apply manual offset
        coords[:, 0] += 30
        coords[:, 1] += 30
        # Ensure coordinates are within bounds
        coords[:, 0] = np.clip(coords[:, 0], 0, patch_size - 1)
        coords[:, 1] = np.clip(coords[:, 1], 0, patch_size - 1)
        # Draw the polygon with the cell type color
        cv2.polylines(patch, [coords], isClosed=True, color=bgr_color, thickness=2)

# Save or display the result
cv2.imwrite("gall-bladder_patch_with_nuclei_colored.png", patch)

Patch image shape: (1024, 1024, 3)


True

In [8]:
import json
import cv2
import numpy as np

# Load the overlayed image
overlayed_image_path = "gall-bladder_patch_with_nuclei_colored.png"
patch = cv2.imread(overlayed_image_path)
if patch is None:
    raise FileNotFoundError(f"Could not load image at {overlayed_image_path}")
print(f"Overlayed image shape: {patch.shape}")

# Load GeoJSON to extract cell type and color information
geojson_path = "/home/KutumLabGPU/Documents/santosh/capstone_project/models/CellViT/example/output/preprocessing/TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311CE/cell_detection/cells.geojson"
with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)

# Extract unique cell types and their colors
cell_type_color_map = {}
for feature in geojson_data:
    if feature["geometry"]["type"] == "MultiPolygon":
        try:
            cell_type = feature["properties"]["classification"]["name"]
            rgb_color = feature["properties"]["classification"]["color"]  # RGB format: [R, G, B]
            # Convert RGB to BGR for OpenCV
            bgr_color = (rgb_color[2], rgb_color[1], rgb_color[0])
            # Add to map if not already present
            if cell_type not in cell_type_color_map:
                cell_type_color_map[cell_type] = bgr_color
        except (KeyError, IndexError):
            # Handle missing classification or color
            cell_type = "Unknown"
            bgr_color = (0, 0, 255)  # Red in BGR
            if cell_type not in cell_type_color_map:
                cell_type_color_map[cell_type] = bgr_color

# Print the cell type color map for debugging
print("Cell type color map:")
for cell_type, color in cell_type_color_map.items():
    print(f"Cell type: {cell_type}, Color (BGR): {color}")

# Add a legend to the image
# Legend parameters
legend_x = patch.shape[1] - 1000  # Bottom-right corner, 250 pixels from the right edge
legend_y = patch.shape[0] - 20 * len(cell_type_color_map) - 20  # Adjust based on number of cell types
legend_width = 300
legend_height = 100 * len(cell_type_color_map) + 10  # 20 pixels per cell type + padding
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1
text_color = (255, 255, 255)  # White text

# Draw a semi-transparent background rectangle for the legend
overlay = patch.copy()
alpha = 0.6  # Transparency factor
cv2.rectangle(overlay, (legend_x, legend_y), (legend_x + legend_width, legend_y + legend_height), (50, 50, 50), -1)  # Gray background
cv2.addWeighted(overlay, alpha, patch, 1 - alpha, 0, patch)

# Draw each cell type and its color in the legend
for idx, (cell_type, color) in enumerate(cell_type_color_map.items()):
    # Draw a colored square
    square_x = legend_x + 10
    square_y = legend_y + 10 + idx * 20
    cv2.rectangle(patch, (square_x, square_y), (square_x + 15, square_y + 15), color, -1)
    
    # Draw the cell type name
    text_x = square_x + 25
    text_y = square_y + 12
    cv2.putText(patch, cell_type, (text_x, text_y), font, font_scale, text_color, font_thickness)

# Save the image with the legend
cv2.imwrite("gall-bladder_patch_with_nuclei_colored_with_legend.png", patch)

Overlayed image shape: (1024, 1024, 3)
Cell type color map:
Cell type: Neoplastic, Color (BGR): (0, 0, 255)
Cell type: Inflammatory, Color (BGR): (77, 221, 34)
Cell type: Connective, Color (BGR): (236, 92, 35)
Cell type: Epithelial, Color (BGR): (68, 159, 255)


True